In [ ]:
import datetime
import re
import time
import requests
import pandas as pd
import spacy
from sqlalchemy import create_engine
from odo import odo

nlp = spacy.load('en_core_web_sm', parser=False)

# Archive.org crawling

In [ ]:
#https://stackoverflow.com/questions/28154066/how-to-convert-datetime-to-integer-in-python
def to_integer(dt_time):
    '''Encode date as monthyear integer, to match archive.org timestamp format'''
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

In [ ]:
#https://stackoverflow.com/questions/993358/creating-a-range-of-dates-in-python
#Build date range of 1500 days back from 12/30/2017
start = datetime.datetime(2017, 12, 30)
dates = [start - datetime.timedelta(days=x) for x in range(0, 1500)]

In [ ]:
#Convert to integer encoding
intdates = [to_integer(date) for date in dates]

In [ ]:
#Capture the point at which A/B testing started
not_testing = []
for i in intdates:
    #Get a snapshot for a given day
    r = requests.get('http://archive.org/wayback/available?url=nytimes.com&timestamp={}'.format(str(i)))
    #Pull the URL for that snapshot from the metadata the API gives us
    arc = r.json()['archived_snapshots']['closest']['url']
    #Scrape the HTML of the snapshot
    r2 = requests.get(arc)
    #Check if the Optimizely script is present. If it is, wait and go to the next day. If not, add it to the list.
    if 'optimizely.com' not in r2.text:
        print(arc)
        not_testing.append(arc)
    else:
        print(i)
        time.sleep(2)

I'm looking for the point where a bunch of URLs show up in the `not_testing` list in a row, then calling the day after the latest date in that run the start point for testing. The latest URL in this run was `http://web.archive.org/web/20150805230843/http://www.nytimes.com:80/`, so testing would have started on August 6, 2015.

# NYT headlines

I'm using the NYT archive API to pull headline data by month. Format: `http://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={your-api-key}`

In [ ]:
key = %env KEY

def query_range(startmo, startyr, endmo, endyr):
    '''Build a range of [month, year] pairs to use for querying the archive API'''
    startmos = [[i, startyr] for i in range(startmo, 13)]
    middle = [[j, i] for i in range(startyr, endyr+1) if i != startyr and i != endyr for j in range(1, 13)]
    endmos = [[i, endyr] for i in range(1, endmo+1)]
    allmos = [i for sub in [startmos, middle, endmos] for i in sub]
    return allmos

def cleaning(entry):
    '''Clean the response of the archive API so that data can be uniformly written into a SQL table'''
    #Check for a print headline
    try:
        entry['print_headline'] = entry['headline']['print_headline']
    except Exception:
        pass
    #Check for a headline
    try:
        entry['headline'] = entry['headline']['main']
    except Exception:
        entry['headline'] = str(entry['headline'])
    #Check for a byline object
    try:
        entry['byline'] = str(entry['byline'])
    except Exception:
        entry['byline'] = ''
    #Turn keywords - returned as a list of objects - into a comma-separated string
    entry['keywords'] = ','.join(i['value'] for i in entry['keywords'])
    #Resolve change over time from 'news_desk' to 'new_desk'
    if 'new_desk' in entry:
        entry['news_desk'] = entry['new_desk']
        entry.pop('new_desk', None)
    #Remove multimedia metadata
    entry.pop('multimedia', None)
    #Remove largely empty blog field
    entry.pop('blog', None)
    #Remove score - unclear purpose
    entry.pop('score', None)
    #Remove inconsistently applied URI
    entry.pop('uri', None)
    return entry

In [ ]:
#Database configuration
user = %env USER
password = %env PASSWORD
db = %env DATABASE
host = %env HOST
engine = create_engine('mysql://{}:{}@{}:3306/{}'.format(user, password, host, db))

In [ ]:
#Generate range of months from mid-2012 to last month
morange = query_range(5, 2012, 9, 2018)

for i in morange:
    #Keep track of current month
    print(i)
    #Query archive API for current month
    h = requests.get('http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}'.format(i[1], i[0], key))
    #Clean results
    items = h.json()['response']['docs']
    cleaned = [cleaning(i) for i in items]
    #Convert results to DataFrame
    cdf = pd.DataFrame(cleaned)
    #Write DataFrame to PostgreSQL table
    cdf.to_sql('results', engine, if_exists='append')

# Intermediate processing

Running all headlines through Spacy NLP, extracting relevant values and storing in separate table with ID. Saves time of having to redo NLP in the future for new features.

In [ ]:
#https://github.com/pandas-dev/pandas/issues/12265#issuecomment-181838631
#Database configuration
user = %env USER
password = %env PASSWORD
db = %env DATABASE
host = %env HOST

df = odo('mysql://{}:{}@{}:3306/{}::results'.format(user, password, host, db), pd.DataFrame)

In [ ]:
def tag_mapping(table):
    table = table[table['headline'].str.len() > 0]
    nlped = table['headline'].apply(lambda x: nlp(x))
    table['tags'] = nlped.apply(lambda x: ' '.join(i.tag_ for i in x))
    table['entities'] = nlped.apply(lambda x: ' '.join(i.text for i in x.ents))
    table['labels'] = nlped.apply(lambda x: ' '.join(i.label_ for i in x.ents))
    dft = table[['_id', 'headline', 'tags', 'entities', 'labels']]
    return dft

In [ ]:
dfnlp = tag_mapping(df)

In [ ]:
dfnlp.to_pickle('saveddf.p')

In [ ]:
odo(dfnlp, 'mysql://{}:{}@{}:3306/{}::nlp'.format(user, password, host, db))

# Generating features

In [ ]:
#Database variables
user = %env USER
password = %env PASSWORD
db = %env DATABASE
host = %env HOST

In [ ]:
#Pipeline for features that don't require external resources 
#(i.e., everything but entities, parts of speech, and LIWC.)
def pipeline(raw):
    '''Combined processing for internally-extracted features'''
    #Define social media phrase regex
    phrases = ['will make you', 
               'this is why', 
               'can we guess', 
               'only [0-9]+ in',
               'the reason is',
               'are freaking out',
               '[0-9]+ stunning photos',
               'tears of joy',
               'is what happens',
               'make you cry',
               'give you goosebumps',
               'talking about it',
               'is too cute',
               'shocked to see',
               'melt your heart',
               '[0-9]+ things only',
               'can\'t stop laughing',
               'top [0-9]+ songs',
               'twitter reacts to',
               'what happened next']
    pattern = '|'.join(phrases)
    
    #Filter out anything that doesn't have a headline
    raw = raw[raw['headline'].str.len() > 0]
    #Clean quotations
    raw['headline'] = raw['headline'].apply(lambda x: re.sub(u'[\u201c\u201d]', '"', x))
    raw['headline'] = raw['headline'].apply(lambda x: re.sub(u'[\u2018\u2019]', '\'', x))
    #Get headline
    hed = raw['headline'].str
    #Count quotations
    raw['quotes'] = hed.count('\'|"')
    #Count numbers
    raw['numbers'] = hed.count('[0-9]+(,[0-9]+)*')
    #Count numbers at the start of a headline
    raw['numbers_starting'] = hed.split(' ').map(lambda x: x[0]).str.count('[0-9]+')
    #Match social pattern
    raw['social_patterns'] = hed.contains(pattern, case = False)
    #Calculate average word length
    raw['avg_word_length'] = hed.split(' ').map(lambda x: sum([len(i) for i in x])/len(x))
    #Get word count
    raw['word_count'] = raw['headline'].map(lambda x: len(x.split(' ')))
    #Get character count
    raw['character_count'] = raw['headline'].map(lambda x: len(x))
    return raw

In [ ]:
dfdd = odo('mysql://{}:{}@{}:3306/{}::results-deduped'.format(user, password, host, db), pd.DataFrame)
dffeat = pipeline(dfdd)

In [ ]:
features = dffeat[['_id', 
                   'word_count', 
                   'quotes', 
                   'numbers', 
                   'numbers_starting', 
                   'social_patterns', 
                   'avg_word_length', 
                   'character_count']]
odo(features, 'mysql://{}:{}@{}:3306/{}::features'.format(user, password, host, db))

## Individual variable definitions

In [ ]:
#H1: Count quotation marks
test['headline'].str.count('\'|"')

In [ ]:
#H2a: Count of numbers
test['headline'].str.count('[0-9]+(,[0-9]+)*')

In [ ]:
#H2b: Count of numbers starting headlines
test['headline'].str.split(' ').map(lambda x: x[0]).str.count('[0-9]+')

In [ ]:
#H3a: Count of interrogatives (WP, WRB) https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
test['headline'].map(lambda x: nlp(x)).map(lambda x: [i.tag_ for i in x]).map(lambda x: x.count('WP') + x.count('WRB'))

In [ ]:
#H3b: Count of interrogatives starting headlines
test['headline'].map(lambda x: nlp(x)).map(lambda x: [i.tag_ for i in x][0]).str.count('WP|WRB')

In [ ]:
#H4: Count of effective social media phrases https://buzzsumo.com/blog/most-shared-headlines-study/#gs.E5zXvW8
test = dfdd.head(n=1000)

phrases = ['will make you', 
           'this is why', 
           'can we guess', 
           'only [0-9]+ in',
           'the reason is',
           'are freaking out',
           '[0-9]+ stunning photos',
           'tears of joy',
           'is what happens',
           'make you cry',
           'give you goosebumps',
           'talking about it',
           'is too cute',
           'shocked to see',
           'melt your heart',
           '[0-9]+ things only',
           'can\'t stop laughing',
           'top [0-9]+ songs',
           'twitter reacts to',
           'what happened next']
pattern = '|'.join(phrases)

test['headline'].str.contains(pattern, case = False)

In [ ]:
#H6: Average word length
test['headline'].str.split(' ').map(lambda x: sum([len(i) for i in x])/len(x))

In [ ]:
#H7a: Count of personal/possessive nouns (PRP, PRP$) https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
test['headline'].map(lambda x: nlp(x)).map(lambda x: [i.tag_ for i in x]).map(lambda x: x.count('PRP') + x.count('PRP$'))

In [ ]:
#H7b: Count of personal/possessive nouns starting headlines
test['headline'].map(lambda x: nlp(x)).map(lambda x: [i.tag_ for i in x][0]).str.count('PRP|PRP$')

In [ ]:
#Word count
test['headline'].map(lambda x: len(x.split(' ')))

In [ ]:
#Character count
test['headline'].map(lambda x: len(x))

## Follow up

In [ ]:
#H8: Count of uncommon words
'''List of popular words taken from https://github.com/dolph/dictionary/blob/master/popular.txt
Disregards cardinal numbers (CD), foreign words (FW), possessive endings (POS), and symbols (SYM/$)'''
#This approach doesn't work very well. Entity analysis?
words = open('popular.txt').read().splitlines()
test['headline'].map(lambda x: nlp(x)) \
    .map(lambda x: [i for i in x if i.tag_ not in ['CD', 'FW', 'POS', 'SYM', '$', '``', ',', ':', 'HYPH', '.']]) \
    .map(lambda x: sum([1 if i.text in words else 0 for i in x]))

In [ ]:
#H9: Count of proper nouns (NNP and NNPS) https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
#Entity analysis?
test['headline'].map(lambda x: nlp(x)).map(lambda x: [i.tag_ for i in x]).map(lambda x: x.count('NNP') + x.count('NNPS'))

In [ ]:
#H10: Count of active verbs


In [ ]:
#H5: Emotional intensity


#### Removing duplicated IDs

In [ ]:
df = odo('mysql://{}:{}@{}:3306/{}::results'.format(user, password, host, db), pd.DataFrame)

In [ ]:
dfnlp = odo('mysql://{}:{}@{}:3306/{}::nlp'.format(user, password, host, db), pd.DataFrame)

In [ ]:
len(df.drop_duplicates('_id').reset_index())

In [ ]:
len(dfnlp.drop_duplicates('_id').reset_index())

In [ ]:
dfdd = df.drop_duplicates('_id').reset_index()
dfdd.drop('level_0', axis=1, inplace=True)
odo(dfdd, 'mysql://{}:{}@{}:3306/{}::results-deduped'.format(user, password, host, db))

In [ ]:
dfnlpdd = dfnlp.drop_duplicates('_id').reset_index()
dfnlpdd.drop('index', axis=1, inplace=True)
odo(dfnlpdd, 'mysql://{}:{}@{}:3306/{}::nlp-deduped'.format(user, password, host, db))